In [86]:
import pandas as pd
%matplotlib inline
import glob
import os
import matplotlib.pyplot as plt
import artm
import numpy as np
from matplotlib.artist import getp
import pandas as pd
import datetime

# размеченные слова

In [2]:
df_with_words = pd.read_csv('./df_with_words.csv' )

In [4]:
df_with_words['score'].value_counts()

1.0    819
0.0    290
Name: score, dtype: int64

# словарь в виде текста

In [102]:
name = 'clean_replace_ent_text_textrank_more_13'
batch_folder = './{}_batches'.format(name)
data_path = './{}_vw.txt'.format(name)
dict_path = batch_folder + '/dictionary.dict'  

dic = artm.Dictionary()

if os.path.isfile(dict_path):
    dic.load(dictionary_path=dict_path)

    batch_vectorizer = artm.BatchVectorizer(data_path=data_path, 
                                    data_format='vowpal_wabbit', 
                                    collection_name='nthhappens', 
                                    target_folder=batch_folder)

In [ ]:
dic.save_text('./test_save_text')

In [5]:
with open('./test_save_text', 'r') as f:
    res = f.read()

res = res.split('\n')

dict_df = pd.DataFrame(list(map(lambda x: x.split(','), res[2:])), columns=res[1].split(',')).dropna()

# замена весов

In [81]:
for_1 = df_with_words[df_with_words['score'] == 1]['word'].values
for_0 = df_with_words[df_with_words['score'] == 0]['word'].values

for_1_mask = dict_df['token'].apply(lambda x: x in for_1)
for_0_mask = dict_df['token'].apply(lambda x: x in for_0)

dict_df.loc[for_1_mask, ' token_value'] = ' -1'
dict_df.loc[for_0_mask, ' token_value'] = ' 1'

In [50]:
dict_df.head()

,token,class_id,token_value,token_tf,token_df
0,hasvalue,@default_class,9.091307333619625e-07,5.0,2.0
1,childcontext,@default_class,9.091307333619625e-07,5.0,1.0
2,voicecommand,@default_class,9.091307333619625e-07,5.0,3.0
3,i_,@default_class,9.091307333619625e-07,5.0,1.0
4,ilumx,@default_class,9.091307333619625e-07,5.0,1.0


# запись текстового словаря

In [84]:
with open('./vocab_semisuper_file.txt', 'w') as f:
    f.write(res[0])
    f.write('\n')
    f.write(res[1])
    f.write('\n')
    for _, row in dict_df.iterrows():
        f.write(','.join(row.values))
        f.write('\n')

In [85]:
dict_df[' token_value'].apply(lambda x: round(float(x)), 4).value_counts()

 0    57332
-1     1506
 1      504
Name:  token_value, dtype: int64

# модель

In [92]:
dictionary = artm.Dictionary()

dictionary.load_text(dictionary_path='./vocab_semisuper_file.txt')

# таким образом, есть 2 словаря dictionary (новый) и dic (старый)

In [164]:
def train_regulars(class_ids, dictionary, num_collection_passes=30, num_document_passes=5, pmt=0.3, seed=0):
    # initialization
    model = artm.ARTM(topic_names=topic_names, cache_theta=True, seed=seed,
                       class_ids=class_ids,
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dictionary)],
                     regularizers=[artm.SmoothSparseThetaRegularizer(
                         name='SparseTheta', tau=-0.5)])    
    # scores
    model.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
    model.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
    model.scores.add(artm.TopicKernelScore(name='TopicKernelScore', topic_names=topic_names,dictionary=dictionary,
                                                  probability_mass_threshold=pmt, class_id='@default_class'))
    model.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=30, 
                                         class_id='@default_class', dictionary=dictionary))
    
    model.scores.add(artm.SparsityPhiScore(name='SparsityScoreModal', class_id='@textrank'))
    #regularizers
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(
                        name='SparsePhi', tau=-0.1, class_ids='@default_class'))
    model.regularizers.add(artm.DecorrelatorPhiRegularizer(
                        name='DecorrelatorPhi', tau=1e+7, class_ids='@default_class'))
    
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(
                        name='SPPhiTextrankReg', tau=-0.2, class_ids='@textrank'))
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(
                        name='SPPhiEntsReg', tau=-0.2, class_ids='@ents'))
    # dictionary
    model.num_document_passes = num_document_passes
    model.initialize(dictionary=dictionary)

    #fit 1
    model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=num_collection_passes)
    print('model fitted', ' ', datetime.datetime.now().time())
    return model

In [98]:
n = 50
topic_names = ['topic_{}'.format(i) for i in range(n)]

In [ ]:
class_ids={'@default_class': 1.0, '@ents': 3.0, '@textrank': 1.5}

In [104]:
model_seed_10_dic = train_regulars(class_ids, dic seed=10, num_collection_passes=50)
model_seed_10_dictionary = train_regulars(class_ids, dictionary, seed=10, num_collection_passes=50)

model fitted   00:03:59.631308


In [105]:
dictionary == dic

False

In [107]:
model_seed_10_dic.get_phi().equals(model_seed_10_dictionary.get_phi())

False

In [108]:
model_seed_10_dic == model_seed_10_dictionary

False

In [109]:
fin = model_seed_10_dic
qwe = []
for topic_name in fin.topic_names:
    qwe.append({'topic_tuned': topic_name, 'top_tuned': fin.score_tracker['TopTokensScore'].last_tokens[topic_name]})

pd.options.display.max_colwidth = 3000

pd.DataFrame(qwe).iloc[:5]

,top_tuned,topic_tuned
0,"[intended, flat, engines, categories, shall, officer, referred, stretch, administrative, please_dont, dpo, wash, free_software, tuning, dude, organisations, fitness, nontechnical, watches, data_subjects, infringement, youre_wrong, washing, unreliable, glibc, permitted, soap, tuned, entrenched, maintainers]",topic_0
1,"[unions, instagram, warnings, engagement, guilty, cynical, likelihood, casual, separation, vitamin, yup, hunting, tactics, systemic, rings, trials, posters, misinformation, jury, punished, pleasure, hollywood, gmo, worldview, helmet, significance, sister, gogs, marijuana, attitudes]",topic_1
2,"[ton, tons, solved, interviews, detect, degrees, resolution, spec, foot, solving, solve_problem, feet, wheel, fiber, steam, tradeoff, electronics, angle, much_harder, whiteboard, metric, jump, velocity, interviewing, discrete, pipe, acceleration, min, interviewer, wheels]",topic_2
3,"[facebook, default, block, instance, essentially, secure, uber, click, requests, googles, info, button, delete, log, map, blog, track, blocking, identity, http, maps, logs, targeted, settings, gmail, blog_post, targeting, handle, cookies, ip_address]",topic_3
4,"[helpful, perform, resource, ruby, decentralized, fork, assumed, centralized, evaluate, assumes, controversial, nontrivial, cognitive, spotify, creator, performed, expressed, actor, artists, optimal, steve, seed, strikes, streams, wellknown, maths, mastodon, opaque, bob, distinct]",topic_4


In [113]:
fin = model_seed_10_dictionary
qwe = []
for topic_name in fin.topic_names:
    qwe.append({'topic_tuned': topic_name, 'top_tuned': fin.score_tracker['TopTokensScore'].last_tokens[topic_name]})

pd.options.display.max_colwidth = 3000

pd.DataFrame(qwe).iloc[:6]

,top_tuned,topic_tuned
0,"[intended, flat, engines, categories, referred, approved, mitigate, administrative, please_dont, dpo, wash, organisations, fitness, nontechnical, data_subjects, infringement, youre_wrong, washing, soap, markup, thrust, entrenched, friend_mine, original_comment, systematic, wtf, conflict_interest, crossed, marketed, borderline]",topic_0
1,"[union, naive, falling, unions, instagram, engagement, cynical, casual, bikes, hunting, wrt, systemic, collectively, trials, moreover, jury, punished, nonexistent, algorithmic, hollywood, criticizing, gmo, fallen, surrounded, helmet, stationary, people_never, crops, never_ever, anxiety]",topic_1
2,"[keeping, style, ton, tons, distance, surface, solved, maintenance, detect, degrees, unknown, solving, latency, metric, spec, foot, feet, fiber, steam, grade, electronics, electrical, whiteboard, stretch, velocity, ford, tuning, acceleration, distances, interviewer]",topic_2
3,"[default, block, instance, essentially, prevent, topic, track, secure, blog, blocking, blocks, brand, http, maps, attacks, settings, blog_post, targeting, ip_address, instances, blocked, pii, handle, vulnerable, ip_addresses, malicious, google_maps, flag, generating, prevents]",topic_3
4,"[perform, resource, consensus, ruby, decentralized, fork, centralized, cognitive, yc, controversial, nontrivial, streams, spotify, performing, performed, expressed, actor, director, artists, steve, highly_recommend, assign, wellknown, kafka, maths, radar, arts, mastodon, opaque, void]",topic_4
5,"[extra, domain, statement, light, head, object, algorithm, algorithms, git, domains, strange, register, id_love, array, constraints, qt, technique, branch, matrix, mathematical, heads, grab, session, finger, registered, clone, pocket, trick, tricks, simulation]",topic_5


In [119]:
fin = model_seed_10_another_dict_plus_reg
qwe = []
for topic_name in fin.topic_names:
    qwe.append({'topic_tuned': topic_name, 'top_tuned': fin.score_tracker['TopTokensScore'].last_tokens[topic_name]})

pd.options.display.max_colwidth = 3000

pd.DataFrame(qwe).iloc[:5]

,top_tuned,topic_tuned
0,"[momentum, certification, judicial, bac, boeing, antenna, consist, dui, nut, taking_account, supervisory, noncompliance, tuners, emdrive, appoint, photon, stay_home, heater, takeoff, xcode, gs, headphone, emitted, risk_management, gloves, disincentive, black_holes, curious_know, emission, carplay]",topic_0
1,"[workable, problem_isnt, harvest, selectively, tire, starving, helmets, gmos, orbits, travelers, monsanto, trickle, soup, amino, hawaii, bath, plea, armchair, deficiencies, ant, golden_rice, amazes, descriptor, xs, unarmed, conflicted, persists, presuming, surroundings, layman]",topic_1
2,"[ball, whiteboard, reserve, nasa, measuring, cm, fractional, xx, hangouts, voltage, inches, apollo, margin_error, whiteboards, hackintosh, rscience, ssn, juggle, grams, increments, mg, distinctions, combustion, monolith, frontpage, bad_behavior, moores_law, open_door, disservice, unification]",topic_2
3,"[vm, https, debian, certificate, vms, firewall, fa, registry, command_line, contribution, reuse, right_way, ec, linux_kernel, gcp, eu_citizen, init, oni, vpns, identifier, replication, pods, cloud_providers, installs, natively, rdf, gvisor, google_amazon, chromeos, curl]",topic_3
4,"[recursion, monad, datalog, b_c, validity, duck, basket, forked, widgets, defer, nim, gsuite, billionaire, alan, digest, forks, dan, interpreters, nobel_prize, cons, trustless, omission, thiel, invariant, ripple, multithreading, graal, hawking, ryan, cryptographically]",topic_4


In [147]:
def train_regulars_both_dicts(class_ids, num_collection_passes=30, num_document_passes=5, pmt=0.3, seed=0):
    # initialization
    model = artm.ARTM(topic_names=topic_names, cache_theta=True, seed=seed,
                       class_ids=class_ids,
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dic),
                       regularizers=[artm.SmoothSparseThetaRegularizer(
                         name='SparseTheta', tau=-0.5)]])    
    # scores
    model.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
    model.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
    model.scores.add(artm.TopicKernelScore(name='TopicKernelScore', topic_names=topic_names,dictionary=dic,
                                                  probability_mass_threshold=pmt, class_id='@default_class'))
    model.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=30, 
                                         class_id='@default_class', dictionary=dic))
    
    model.scores.add(artm.SparsityPhiScore(name='SparsityScoreModal', class_id='@textrank'))
    #regularizers
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(
                         name='SmoothSparsePhiRegularizer', dictionary=dictionary))
        
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(
                        name='SparsePhi', tau=-0.1, class_ids='@default_class'))
    model.regularizers.add(artm.DecorrelatorPhiRegularizer(
                        name='DecorrelatorPhi', tau=1e+7, class_ids='@default_class'))
    
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(
                        name='SPPhiTextrankReg', tau=-0.2, class_ids='@textrank'))
    model.regularizers.add(artm.SmoothSparsePhiRegularizer(
                        name='SPPhiEntsReg', tau=-0.2, class_ids='@ents'))
    # dictionary
    model.num_document_passes = num_document_passes
    model.initialize(dictionary=dic)

    #fit 1
    model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=num_collection_passes)
    print('model fitted', ' ', datetime.datetime.now().time())
    return model

In [115]:
model_seed_10_both = train_regulars_both_dicts(class_ids, seed=10, num_collection_passes=50)


model fitted   00:34:39.312466


#### пример из туториала


model.regularizer.add(
    artm.SmoothSparsePhiRegularizer(name='smooth_sparse_phi_regularizer', dictionary=my_dictionary)
)

In [166]:
model_seed_10_otkatis.score_tracker['PerplexityScore'].value

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [168]:
models = [model_seed_10, model_seed_10_2, model_seed_10_another_dict, model_seed_10_another_dict_plus_reg, model_seed_10_experiment, model_seed_10_otkatis]

In [151]:
# Функция для печати значений метрик нескольких моделей
def print_measures_models(models):
    print('Sparsity Phi: {}'.format('    '.join([
        str(model_artm.score_tracker['SparsityPhiScore'].last_value) for model_artm in models])))

    print('Sparsity Theta: {}'.format('    '.join([
        str(model_artm.score_tracker['SparsityThetaScore'].last_value) for model_artm in models])))

    print('Kernel contrast: {}'.format('    '.join([
        str(model_artm.score_tracker['TopicKernelScore'].last_average_contrast) for model_artm in models])))

    print('Kernel purity: {}'.format('    '.join([
        str(model_artm.score_tracker['TopicKernelScore'].last_average_purity) for model_artm in models])))
    
    print('Kernel size: {}'.format('    '.join([
        str(model_artm.score_tracker['TopicKernelScore'].last_average_size) for model_artm in models])))

    print('Perplexity: {}'.format('    '.join([
        str(model_artm.score_tracker['PerplexityScore'].last_value) for model_artm in models])))

In [161]:
model_seed_10.get_theta()[model_seed_10.get_theta()[122005]!=0]

,122000,122001,122002,122003,122004,122005,122006,122007,122008,122009,...,63990,63991,63992,63993,63994,63995,63996,63997,63998,63999
topic_13,0.000000,0.166709,0.0,0.000000,0.000000,0.191174,0.000000,0.000000,0.000000,0.000000,...,0.0,0.051753,0.02649,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.100174
topic_26,0.205665,0.000000,0.0,0.028832,0.054137,0.095847,0.066456,0.159681,0.123966,0.000000,...,0.0,0.000000,0.00000,0.088216,0.000000,0.0,0.0,0.0,0.000000,0.102929
topic_31,0.000000,0.000000,0.0,0.000000,0.000000,0.341466,0.004767,0.039468,0.000000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.019521,0.0,0.0,0.0,0.000000,0.000000
topic_34,0.000000,0.089462,0.0,0.000000,0.000000,0.085991,0.000000,0.000000,0.000000,0.000000,...,0.0,0.024580,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.101725,0.000000
topic_39,0.000000,0.072272,0.0,0.000000,0.000000,0.285523,0.000000,0.000000,0.000000,0.066676,...,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [169]:
print_measures_models(models)

Sparsity Phi: 0.9751600623130798    0.9761093258857727    0.9525267481803894    0.9933470487594604    0.9913120865821838    0.9525267481803894
Sparsity Theta: 0.7829350829124451    0.7902418375015259    0.7855408191680908    0.0    0.018507836386561394    0.7855413556098938
Kernel contrast: 0.8609730005264282    0.8598587512969971    0.766845703125    0.9998452067375183    0.9004098773002625    0.7668805718421936
Kernel purity: 0.5702741742134094    0.7859736680984497    0.8402527570724487    0.9999809861183167    0.9699714779853821    0.8402342200279236
Kernel size: 4010.43994140625    4020.60009765625    926.1400146484375    304.3999938964844    1782.219970703125    926.0999755859375
Perplexity: 1577.3531494140625    1057.6279296875    nan    nan    1042.97314453125    nan


In [121]:
model_seed_10_another_dict_plus_reg.score_tracker['PerplexityScore'].value

[5863.5107421875,
 1244.486328125,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [127]:
(model_seed_10_another_dict_plus_reg.get_phi() == 0).mean()

topic_0     0.991726
topic_1     0.992653
topic_2     0.991591
topic_3     0.987951
topic_4     0.990344
topic_5     0.987092
topic_6     0.992299
topic_7     0.989586
topic_8     0.988271
topic_9     0.992720
topic_10    0.991321
topic_11    0.990732
topic_12    0.987193
topic_13    0.991574
topic_14    0.987698
topic_15    0.991541
topic_16    0.990041
topic_17    0.989653
topic_18    0.990816
topic_19    0.989502
topic_20    0.975869
topic_21    0.991018
topic_22    0.987142
topic_23    0.990732
topic_24    0.991861
topic_25    0.990513
topic_26    0.990361
topic_27    0.989889
topic_28    0.989805
topic_29    0.988524
topic_30    0.993748
topic_31    0.992114
topic_32    0.991878
topic_33    0.986064
topic_34    0.992569
topic_35    0.990327
topic_36    0.992029
topic_37    0.988878
topic_38    0.989923
topic_39    0.990378
topic_40    0.992333
topic_41    0.990277
topic_42    0.992686
topic_43    0.990664
topic_44    0.991001
topic_45    0.990209
topic_46    0.990934
topic_47    0

In [129]:
(model_seed_10_another_dict_plus_reg.get_phi()).sum()

topic_0     3.000000
topic_1     3.000000
topic_2     3.000003
topic_3     2.999996
topic_4     3.000003
topic_5     3.000005
topic_6     2.999998
topic_7     2.999998
topic_8     2.999995
topic_9     2.999998
topic_10    2.999998
topic_11    3.000004
topic_12    3.000002
topic_13    2.999996
topic_14    3.000001
topic_15    2.999994
topic_16    2.999991
topic_17    2.999995
topic_18    3.000001
topic_19    2.999997
topic_20    3.000005
topic_21    2.999999
topic_22    2.999991
topic_23    3.000000
topic_24    2.999999
topic_25    3.000002
topic_26    2.999999
topic_27    2.999997
topic_28    3.000004
topic_29    2.999995
topic_30    2.999997
topic_31    2.999997
topic_32    3.000001
topic_33    2.999998
topic_34    3.000001
topic_35    3.000000
topic_36    3.000000
topic_37    2.999990
topic_38    2.999990
topic_39    2.999998
topic_40    3.000004
topic_41    3.000000
topic_42    2.999996
topic_43    2.999991
topic_44    3.000003
topic_45    3.000000
topic_46    3.000001
topic_47    2

In [132]:
model_seed_10_another_dict_plus_reg.get_theta()

,122000,122001,122002,122003,122004,122005,122006,122007,122008,122009,...,63990,63991,63992,63993,63994,63995,63996,63997,63998,63999
topic_0,0.039677,0.013600,0.039632,0.019337,0.034247,0.015873,0.013848,0.063715,0.014760,0.013246,...,0.015038,0.018227,0.012988,0.012983,0.013333,0.013334,0.013407,0.027778,0.037597,0.015782
topic_1,0.014085,0.019011,0.022350,0.036382,0.034247,0.087302,0.010103,0.021352,0.013245,0.020192,...,0.015038,0.011502,0.017707,0.011429,0.013333,0.016988,0.021071,0.013889,0.052238,0.013889
topic_2,0.014089,0.011838,0.009582,0.036276,0.013699,0.015873,0.010110,0.013991,0.013249,0.013245,...,0.015063,0.011519,0.017961,0.038579,0.017887,0.033410,0.011779,0.035913,0.015038,0.013894
topic_3,0.015872,0.013599,0.015977,0.019339,0.013699,0.015873,0.013852,0.039818,0.029521,0.013254,...,0.015038,0.045356,0.012987,0.012986,0.013333,0.013333,0.013405,0.013889,0.015038,0.015783
topic_4,0.014085,0.011834,0.013150,0.014493,0.013699,0.015873,0.010115,0.019192,0.013245,0.017892,...,0.015073,0.011569,0.012987,0.011552,0.013333,0.013333,0.016768,0.013889,0.015038,0.013894
topic_5,0.050603,0.024395,0.014806,0.015376,0.027397,0.039683,0.127136,0.014413,0.013565,0.013266,...,0.030094,0.018233,0.016488,0.035100,0.041406,0.013333,0.042459,0.016811,0.015038,0.043214
topic_6,0.014085,0.021856,0.014768,0.014493,0.013699,0.015873,0.010350,0.013986,0.020213,0.013245,...,0.016817,0.029506,0.017257,0.011484,0.013333,0.013333,0.039481,0.013889,0.015038,0.014118
topic_7,0.039679,0.013599,0.016352,0.019339,0.013699,0.015873,0.013951,0.015928,0.014761,0.026490,...,0.015039,0.018731,0.012987,0.032533,0.033333,0.013333,0.013412,0.013889,0.015038,0.015782
topic_8,0.014085,0.011834,0.009569,0.014493,0.013699,0.015873,0.010101,0.013986,0.013245,0.013245,...,0.015038,0.011494,0.017979,0.057377,0.017903,0.013333,0.011765,0.044924,0.015038,0.013889
topic_9,0.035211,0.011834,0.009575,0.028986,0.013699,0.015873,0.010103,0.034966,0.013245,0.013245,...,0.015047,0.011518,0.012987,0.011506,0.013334,0.013333,0.023537,0.013889,0.015038,0.013889


In [130]:
model_seed_10_another_dict_plus_reg.info

config {
  topic_name: "topic_0"
  topic_name: "topic_1"
  topic_name: "topic_2"
  topic_name: "topic_3"
  topic_name: "topic_4"
  topic_name: "topic_5"
  topic_name: "topic_6"
  topic_name: "topic_7"
  topic_name: "topic_8"
  topic_name: "topic_9"
  topic_name: "topic_10"
  topic_name: "topic_11"
  topic_name: "topic_12"
  topic_name: "topic_13"
  topic_name: "topic_14"
  topic_name: "topic_15"
  topic_name: "topic_16"
  topic_name: "topic_17"
  topic_name: "topic_18"
  topic_name: "topic_19"
  topic_name: "topic_20"
  topic_name: "topic_21"
  topic_name: "topic_22"
  topic_name: "topic_23"
  topic_name: "topic_24"
  topic_name: "topic_25"
  topic_name: "topic_26"
  topic_name: "topic_27"
  topic_name: "topic_28"
  topic_name: "topic_29"
  topic_name: "topic_30"
  topic_name: "topic_31"
  topic_name: "topic_32"
  topic_name: "topic_33"
  topic_name: "topic_34"
  topic_name: "topic_35"
  topic_name: "topic_36"
  topic_name: "topic_37"
  topic_name: "topic_38"
  topic_name: "topic_39"
 

In [122]:
model_seed_10.score_tracker['PerplexityScore'].value

[37959.32421875,
 2047.3466796875,
 1756.7103271484375,
 1372.012451171875,
 1373.1568603515625,
 1240.6375732421875,
 1176.43115234375,
 1147.5987548828125,
 1127.3853759765625,
 1114.9215087890625,
 1104.9720458984375,
 1097.9609375,
 1092.1961669921875,
 1087.893798828125,
 1084.22021484375,
 1081.2933349609375,
 1078.6639404296875,
 1076.503173828125,
 1074.533203125,
 1072.9180908203125,
 1071.4608154296875,
 1070.26904296875,
 1069.1663818359375,
 1068.2724609375,
 1067.39697265625,
 1066.673828125,
 1065.91845703125,
 1065.32373046875,
 1064.664306640625,
 1064.1866455078125,
 1063.6236572265625,
 1063.1927490234375,
 1062.6983642578125,
 1062.3052978515625,
 1061.8583984375,
 1061.5108642578125,
 1061.115478515625,
 1060.8018798828125,
 1060.436279296875,
 1060.1495361328125,
 1059.8116455078125,
 1059.5484619140625,
 1059.2388916015625,
 1058.999267578125,
 1058.7135009765625,
 1058.495849609375,
 1058.2352294921875,
 1058.043212890625,
 1057.8062744140625,
 1057.628662109375]